In [0]:
# ! pip install chatterbot 
# ! pip install chatterbot_corpus

In [0]:
# ! pip install --upgrade chatterbot 
# ! pip install --upgrade chatterbot_corpus

# Import Statements

In [0]:
import warnings
warnings.filterwarnings('ignore')

# Import statements
import pandas as pd
import numpy as np
import pprint as pp
import json
from pandas.io.json import json_normalize
import re
from timeit import default_timer
from  more_itertools import unique_everseen

# Preprocessing
# from pycontractions import Contractions

# Tokenization imports
import nltk
# from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

# Puncuation & lower case
import string #punctuation removal

# Stop words
from nltk.corpus import stopwords

# Lemmatizer
from nltk.stem.wordnet import WordNetLemmatizer

# POS tagging
from nltk.corpus import wordnet

# NER
import nltk
import spacy
import pprint as pprint
import en_core_web_sm # CNN gets loaded in, sees what words depends on each other, POS tagging, entity recognition 
from spacy import displacy # Visualize NER

# Chatterbot
# from chatterbot import ChatBot
# from chatterbot.trainers import ListTrainer

# Data Preprocessing

In [0]:
# This is testing with part 1 - 10% of data
with open('/content/drive/My Drive/contraction_data_parts/expand_convo_text_1.txt', 'r') as file:
    convo_all = file.read()

In [3]:
# Let's see the text data, seems messy 
# When we extracted conversations from the frames dataset, we split every statement with a new line and every conversations with *
# Through the use of pycontractions, every new line (\n) was added with an extra \
pp.pprint(convo_all[0:1000]) 

('["I would like to book a trip to Atlantis from Caprica on Saturday, August '
 '13, 2016 for 8 adults. I have a tight budget of 1700.\\nHi...I checked a few '
 'options for you, and unfortunately, we do not currently have any trips that '
 'meet this criteria.  Would you like to book an alternate travel '
 'option?\\nYes, how about going to Neverland from Caprica on August 13, 2016 '
 'for 5 adults. For this trip, my budget would be 1900.\\nI checked the '
 'availability for this date and there were no trips available.  Would you '
 'like to select some alternate dates?\\nI have no flexibility for dates... '
 'but I can leave from Atlantis rather than Caprica. How about that?\\nI '
 'checked the availability for that date and there were no trips available.  '
 'Would you like to select some alternate dates?\\nI suppose I will speak with '
 'my husband to see if we can choose other dates, and then I will come back to '
 'you.Thanks for your help\\n**************************************

In [0]:
# Splits text data into separate lists for each conversation
def text_to_convo(text):
  list_convos = [convo.split('\n') for convo in text.split('*') if text]         # Conversation delimited by *
  list_convos = [convo for convo in list_convos if convo != ['']]                # Remove empty conversation
  return list_convos

In [0]:
# Splits all the statements from a conversation into their own string
def convo_to_statement(list_convos):
  sep_convos = []
  for items in list_convos: 
    sep_convos.append([])                     # Creates list for each conversation
    for item in items:
      item = item.split('\\n')                # Splits conversations into statements 
      sep_convos[-1].append(item)             # Adds all statements to corresponding list
  return sep_convos

In [0]:
# Flattens the list as there is a list in another list
# - Handles sentence tokenization, as all statements are separated into their own string in this list 
def flatten_list(conversation): 
  flat_list = []
  for sublist in conversation:
    for item in sublist:
          flat_list.append(item)
  return flat_list

In [0]:
# Extracts only the users' statements from the conversations
def extract_user_statement(conversation):
  user_statement = conversation[0::2] 
  return user_statement

## NER

In [0]:
# %pip install spacy -qq 

In [0]:
# English multi-task CNN trained on OntoNotes. Assigns context-specific token vectors, POS tags, dependency parse and named entities
def ner_model(sample_conversation):
  nlp = en_core_web_sm.load()
  text = ' '.join(sample_conversation)
  convo_nlp = nlp(text)
  return convo_nlp

In [0]:
# Renders a dependency parse tree or named entity visualization
def ner_model_viz(document):
  viz = displacy.render(document, jupyter=True, style='ent', page=True)
  return viz

In [0]:
# Extracts locations from NER model
def extract_location_ner(nlp_doc):
  gpe_list = []
  for entity in nlp_doc.ents:
    if entity.label_ == 'GPE':      #GPE -> Countries, cities, states
      gpe_list.append(entity.text)
      
  return list(unique_everseen(gpe_list))

In [0]:
# Extracts dates from NER model 
def extract_date_ner(nlp_doc):
  date_list = []
  for entity in nlp_doc.ents:
    if entity.label_ == 'DATE':      #DATE -> Absolute or relative dates or periods
      date_list.append(entity.text)
      
  return date_list

In [0]:
# Extracts dates from NER model 
def extract_price_ner(nlp_doc):
  price_list = []
  for entity in nlp_doc.ents:
    if entity.label_ == 'MONEY':      #Money -> Monetary values, including unit
      price_list.append(entity.text)
    elif entity.label == 'CARDINAL':
      price_list.append(entity.text)
  return price_list

In [0]:
def ner_ouput(location_list,date_list,price_list):
  output = {'Locations':location_list,
            'Dates':date_list,
            'Price':price_list}
  return output

# Main

In [0]:
# Preprocess frames data set using functions above 
def data_preprocess(text_data):
  list_of_convo = text_to_convo(text_data) 
  list_of_statement = convo_to_statement(list_of_convo)
  return list_of_statement

In [0]:
# Preprocess a conversation using all functions above
def convo_preprocess(conversation):
  flat_convo = flatten_list(conversation)
  list_user_statement = extract_user_statement(flat_convo)
  # list_of_word = word_token(list_user_statement)
  # no_punct_word = punc_removal(list_of_word)
  # lower_word = word_to_lowercase(no_punct_word)
  # no_stop_word = stop_word_removal(lower_word)
  # lem_word = word_lemmatization(no_stop_word)
  return list_user_statement

In [0]:
# Conversations
all_convo = data_preprocess(convo_all) # from frames dataset
convo_1  = [['I want to go from Toronto to New York from August 1 to August 15. My budget is $800']]
convo_2 = all_convo[4] # from frames dataset
convo_3 = all_convo[10] # from frames dataset

### Sample Conversation 1

In [57]:
# Preprocessing
sample_convo = convo_preprocess(convo_1)
pp.pprint(convo_1)

# Applying preprocessed text to NER
nlp_doc = ner_model(sample_convo)
ner_viz = ner_model_viz(nlp_doc)

# Extracting information from NER model
extracted_location = extract_location_ner(nlp_doc)
extracted_date = extract_date_ner(nlp_doc)
extracted_price = extract_price_ner(nlp_doc)

ner_ouput(extracted_location, extracted_date, extracted_price)

[['I want to go from Toronto to New York from August 1 to August 15. My budget '
  'is $800']]


{'Dates': ['August 1 to August 15'],
 'Locations': ['Toronto', 'New York'],
 'Price': ['800']}

### Sample Conversation 2

In [0]:
# Preprocessing
sample_convo = convo_preprocess(convo_2)
pp.pprint(convo_2)

# Applying preprocessed text to NER
nlp_doc = ner_model(sample_convo)
ner_viz = ner_model_viz(nlp_doc)

# Extracting information from NER model
extracted_location = extract_location_ner(nlp_doc)
extracted_date = extract_date_ner(nlp_doc)
extracted_price = extract_price_ner(nlp_doc)

ner_ouput(extracted_location, extracted_date, extracted_price)

[['Hello, I am looking to book a trip for 2 adults and 6 children for $21,300 '
  'or less. We are departing from Kochi for Denver.',
  'I have several options available within your budget. How long would you '
  'like to travel for? And do you have dates in mind?',
  'I do not have any dates in mind. I would like to spend as much time in '
  'Denver as my budget will allow.',
  'I can book 7 days at a 4.0 star hotel for 19028.93USD. I can also book 7 '
  'days at a 3.0 star hotel for 12824.84USD.',
  'Do these packages have different departure dates? When would I be leaving '
  'for each of them?',
  'The 3.0 star trip leaves Kochi August 26 and returns August 31. The 4.0 '
  'star leaves August 27 from Kochi and returns September 1.',
  'Ok, I would like to purchase the trip with the 4-star hotel.',
  'Perfect, I will book that trip for you now. You will depart Kochi at 9:00 '
  'am August 27. Have a nice day.',
  'Thank you',
  '']]


{'Dates': [], 'Locations': ['kochi', 'denver'], 'Price': ['21300']}

### Sample Conversation 3

In [0]:
# Preprocessing
sample_convo = convo_preprocess(convo_3)
pp.pprint(convo_3)

# Applying preprocessed text to NER
nlp_doc = ner_model(sample_convo)
ner_viz = ner_model_viz(nlp_doc)

# Extracting information from NER model
extracted_location = extract_location_ner(nlp_doc)
extracted_date = extract_date_ner(nlp_doc)
extracted_price = extract_price_ner(nlp_doc)

ner_ouput(extracted_location, extracted_date, extracted_price)

[['I would like to get away from Monday, August 15, 2016 to Wednesday, August '
  '31, 2016. My budget is $3200 and I would leave from Detroit.',
  'And what destination would you like to travel to?',
  'Anywhere, I am looking for an adventure. What do you have?',
  'How about Mexico or Porto Alegre?',
  'let us look at both',
  'Mexico is unavailable.  Porto Alegre is available, but you would have to '
  'leave from Santos.',
  'Can you please suggest some destinations that are available? I am leaving '
  'from Detroit.',
  'I have no departures available from Detroit. for these dates.  Would you '
  'like to make a date change?',
  'Anything between August 15 and August 31?',
  'I am sorry, but we do not have any departures from Detroit available for '
  'these dates for the budget you provided.  Would you like to adjust your '
  'budget for this trip?',
  'No I cannot. Thank you for your help',
  '']]


{'Dates': ['monday august 15 2016',
  'wednesday',
  'august 31',
  'august 15 august 31'],
 'Locations': ['detroit'],
 'Price': ['3200']}

# Chatterbot

## Next Steps:

In [0]:
# Create a new chat bot named Charlie
chatbot = ChatBot('Charlie')
trainer = ListTrainer(chatbot)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
trainer.train([
    'How are you?',
    'I am good.',
    'That is good to hear.',
    'Thank you',
    'You are welcome.',
])

List Trainer: [####################] 100%


In [0]:
trainer.train(flat_list)

List Trainer: [####################] 100%


In [0]:
print(chatbot.get_response('Hi'))

To start, just give me some information on where you would like to travel, your budget, your point of departure, or any other travel info you can think of.


In [0]:
print(chatbot.get_response('I would like to travel from Toronto to Japan'))

Did you have any specific dates in mind?


In [0]:
print(chatbot.get_response('June to August'))

How much would that be?


In [0]:
print(chatbot.get_response('under 1200'))

Perfect. Thank you.


# Amadeus

In [20]:
extracted_location


['Toronto', 'Tokyo']

In [22]:
extracted_location[1]

'Tokyo'

In [19]:
# extracted_date 
extracted_price 


['3821']

In [81]:
"""

Travel Chat Bot for Slack
title:  amadeus_query.py
author: George Thoraninh
usage:  python amadeus_query.py

"""
# import statements
import os
import ssl
import requests
from functools import lru_cache
import time
from datetime import datetime
from dateutil import parser




########### 
# Amadeus #
########### 
amadeus_api_key = os.environ['AMADEUS_API_KEY']
amadeus_api_secret = os.environ['AMADEUS_API_SECRET']


auth_response = requests.post('https://api.amadeus.com/v1/security/oauth2/token',
                              data={'grant_type': 'client_credentials',
                                    'client_id': amadeus_api_key,
                                    'client_secret': amadeus_api_secret})
bearer = auth_response.json()['access_token']
#print(bearer)
# API has quotas, so better to cache everything
@lru_cache(maxsize=2048)
def call_api(url, **params):
    full_url = f'https://api.amadeus.com/v1{url}'
    response = requests.get(full_url,
                            params=params,
                            headers={'Authorization': f'Bearer {bearer}',
                                     'Content-Type': 'application/vnd.amadeus+json'})
    return response.json()


KeyError: ignored

In [0]:
airport_df = pd.read_csv('clean_airports.csv')

In [0]:
def location_to_iata(extracted_location):
  '''
  Converts the city location to airport code 
  used for API call. 
  Returns results as a tuple.
  '''
  origin_place = extracted_location[0] #Fix: parse better, multiple locations?
  destination_place = extracted_location[1]

  origin_iata = airport_df[airport_df['City']==origin_place]['IATA'].iloc[0] # Fix: lower/upper case
  destination_iata = airport_df[airport_df['City']==destination_place]['IATA'].iloc[0]
  return origin_iata, destination_iata #

In [0]:
def format_date(extracted_date):
  '''
  Converts the date to the format: YYYY-MM-DD 
  Returns results as a tuple.
  '''
  date_string = extracted_date[0]
  date_list = date_string.split('to') # Fix: other date inputs
  datetime_depart = parser.parse(date_list[0]).strftime('%Y-%m-%d')
  datetime_return = parser.parse(date_list[1]).strftime('%Y-%m-%d')
  return datetime_depart, datetime_return 

In [79]:
type(extracted_price[0])

str

In [0]:
def flight_offer_get(extracted_location, extracted_date, extracted_price):
  
  # Retrieves the necessary location information
  iata_tuple = location_to_iata(extracted_location)
  origin_iata = iata_tuple[0]
  destination_iata = iata_tuple[1]

  # Retrieves the necessary date information
  date_tuple = format_date(extracted_date)
  depart_date = date_tuple[0]
  return_date = date_tuple[1]

  response = call_api('/shopping/flight-offers',
                  origin = origin_iata,
                  destination = destination_iata,
                  departureDate = depart_date,
                  returnDate = return_date,
                  adults='1', # Set ticket search for one adult
                  currency='CAD', # Set currency as CAD 
                  maxPrice = extracted_price[0],
                  max='5' # Set the limit of results returned
                  )
  return response